# CUDA Accelerated Gen Distributions

In [89]:
using BenchmarkTools
using Gen
using CUDA
push!(LOAD_PATH, ENV["probcomp"]*"/Gen-Distribution-Zoo/src")
using GenDistributionZoo: ProductDistribution, diagnormal
push!(LOAD_PATH,"src")
using MyUtils

In [251]:
#nbx
"""
    griddims = cuda_grid(datadims::Tuple{Vararg{Int}}, 
                         blockdims::Tuple{Vararg{Int}})

Given data dimensions `datadims` and number of threads 
in each dimension `blockdims` returns the respective 
grid dimensions `griddims` such that

    griddims[i] = ceil(Int, datadims[i]/blockdims[i])

"""
function cuda_grid(datadims::Tuple{Vararg{Int}}, blockdims::Tuple{Vararg{Int}})
    griddims = ceil.(Int, datadims./blockdims)
    return griddims
end

cuda_grid

In [73]:
remove_dim(t::Tuple, dim::Int) = (t[1:dim-1]...,t[dim+1:end]...)
remove_dims(t::Tuple, dims::Tuple{Vararg{Int}}) = t[filter(x-> !(x in dims), 1:length(t))]
insert_dim(t::Tuple, dim::Int) = (t[1:dim-1]...,1,t[dim:end]...)

function squeeze_dim(x, dim::Int)
    @assert size(x, dim) == 1 "Can't sqeeze dim that is bigger than one. We have \"size(x, dim)=$(size(x, dim))\""
    return reshape(x, remove_dim(size(x), dim))
end
    
function squeeze_dims(x, dims::Tuple{Vararg{Int}})
    @assert all(size.([x], dims) .== 1) "Can't sqeeze dim that is bigger than one"
    return reshape(x, remove_dims(size(x), dims))
end
    
function unsqueeze_dim(x, dim::Int)
    return reshape(x, insert_dim(size(x), dim))
end

function match_shape(x, dim::Int)
    shape = fill(1, ndims(x))
    shape[dim] = size(x,dim)
    return Tuple(shape)
end

match_shape (generic function with 2 methods)

In [66]:
w = rand(2)
x = rand(1,2,3)
match_shape(w, x, 2)

1×2×1 Array{Float64, 3}:
[:, :, 1] =
 0.062176  0.802435

In [5]:
d = 3
t = (1,2,3,4,5,6)
@assert t == remove_dim(insert_dim(t, d),d)
@btime remove_dim($((1,2,3,4,5,6,7)),3);
@btime insert_dim($((1,2,3,4,5,6,7)),3);
@btime squeeze_dim($(rand(10,10,1,10)), 3);

dims = (5,6)
@btime remove_dims($t, $dims);

  347.673 ns (3 allocations: 144 bytes)
  423.156 ns (3 allocations: 160 bytes)
  304.739 ns (5 allocations: 176 bytes)
  422.196 ns (4 allocations: 352 bytes)


## Gaussian

In [221]:
function gaussian_logpdf(x, mu, sig)
    d = (x .- mu).^2 ./ sig.^2
    log_p = - log.(sqrt.(sig * 2 * π)) .- 1/2 * d
    return log_p
end;

Reality check:

In [222]:
m = 500 # num mixture components
n = 500 # num observations

sig = ones(1,m,2)
mu  = zeros(1,m,2) 
x   = rand(n,1,2)

isapprox(
    gaussian_logpdf(x, mu, sig),
    logpdf.([normal],x,mu,sig)
)

true

Benchmark

In [232]:
for (m,n) in [(1,100),(100,100),(60,360),(360,360)]

    sig = ones(1,m,2)
    mu  = zeros(1,m,2) 
    x   = rand(n,1,2)

    sig_ = CuArray(sig)
    mu_  = CuArray(mu)
    x_   = CuArray(x)
    
    println("\nm=$(m), n=$(n)")
    @btime gaussian_logpdf($x,  $mu,  $sig)  samples=3 evals=3;
    @btime gaussian_logpdf($x_, $mu_, $sig_) samples=3 evals=3;
end


m=1, n=100
  2.069 μs (7 allocations: 5.61 KiB)
  104.798 μs (200 allocations: 14.44 KiB)

m=100, n=100
  93.713 μs (10 allocations: 476.00 KiB)
  104.059 μs (203 allocations: 14.48 KiB)

m=60, n=360
  184.585 μs (10 allocations: 1016.94 KiB)
  103.283 μs (203 allocations: 14.48 KiB)

m=360, n=360
  1.066 ms (10 allocations: 5.96 MiB)
  108.153 μs (207 allocations: 14.55 KiB)


```julia
m=1, n=100
  2.069 μs (7 allocations: 5.61 KiB)
  104.798 μs (200 allocations: 14.44 KiB)

m=100, n=100
  93.713 μs (10 allocations: 476.00 KiB)
  104.059 μs (203 allocations: 14.48 KiB)

m=60, n=360
  184.585 μs (10 allocations: 1016.94 KiB)
  103.283 μs (203 allocations: 14.48 KiB)

m=360, n=360
  1.066 ms (10 allocations: 5.96 MiB)
  108.153 μs (207 allocations: 14.55 KiB)
```

## Mixture

In [234]:
function mixture(ws, log_ps)
    @assert length(ws) == length(log_ps)
    return reduce(.+, ws .* log_ps)
end

mixture (generic function with 1 method)

In [235]:
m = 20 # num mixture components
n = 300 # num observations

sig = ones(1,m,2)
mu  = zeros(1,m,2) 
x   = rand(n,1,2)

log_p_ = gaussian_logpdf(x_, mu_, sig_)
log_p  = gaussian_logpdf(x, mu, sig)


ws = [0.5, 0.5]



log_ps  = [log_p, log_p]
log_ps_ = [log_p_, log_p_]


@btime mixture(ws, log_ps)  samples=3 evals=3;
@btime mixture(ws, log_ps_) samples=3 evals=3;

  47.512 μs (7 allocations: 281.50 KiB)
  41.607 μs (88 allocations: 6.17 KiB)


In [236]:
function mixture_along_dim(log_p, w; dim)
    @assert length(w) == size(log_p, dim)
    
    shape = match_shape(log_p, dim)
    w     = reshape(w, shape)
    log_p = log.(sum(exp.(log_p .+ log.(w)), dims=dim)) 
    
    return squeeze_dim(log_p, dim) 
end

mixture_along_dim (generic function with 2 methods)

In [237]:
m =   7 # num mixture components
n = 100 # num observations

gm = HomogeneousMixture(normal, [0,0])

mu  = rand(n,m); 
sig = ones(n,m);
x   = rand(n);

mu_  = CuArray(mu)
sig_ = CuArray(sig)
x_   = reshape(CuArray(x), n,1)

isapprox( 
    [logpdf(gm, x[i], fill(1/m, m), mu[i,:], sig[i,:]) for i=1:n],
    Array(mixture_along_dim(gaussian_logpdf(x_, mu_, sig_), CUDA.fill(1/m,m), dim=2))
)

true

In [238]:
m = 500 # num mixture components
n = 500 # num observations

sig = ones(1,m)
mu  = zeros(1,m) 
x   = rand(n,1)
w = fill(1/m,m)

sig_ = CuArray(sig)
mu_  = CuArray(mu)
x_   = CuArray(x)
w_ = CuArray(w)

function bench_mixture(x,mu,sig,w)
    mixture_along_dim(gaussian_logpdf(x, mu, sig),w, dim=2);
end

@btime bench_mixture($x,$mu,$sig,$w)     samples=3 evals=3;
@btime bench_mixture($x_,$mu_,$sig_,$w_) samples=3 evals=3;

  7.666 ms (35 allocations: 7.65 MiB)
  187.444 μs (335 allocations: 20.23 KiB)


## Sensor Mixture Product

$$
    \prod_i \sum_j w_j \cdot p_{i,j}(x_i) 
$$

$$
p_{i,1} = \sum_{k=1}^m \tfrac{1}{m} N(x_i ; y_{i,k}, \sigma)
$$

$$
p_{i,2}(x_i) \equiv \text{const}
$$

In [273]:
n = 360
m = 20
x = rand(n)
y = rand(n,2*m+1)

x_ = CuArray(x)
y_ = CuArray(y)

function slw(x, w::Int, y)    
    
#     for i=-w:w
#        y[:,i+w+1] = circshift(z,i)
#     end
    y = reduce(hcat, (circshift(x,s) for s=-w:w))
    return y
end

@btime slw($x,$m,$y) samples=3 evals=3;

  542.955 μs (117 allocations: 2.48 MiB)


In [284]:
function sliding_window_cpu!(x, w::Int, y)   
    for i = 1:size(x,1), j = 1:2*w+1
        @inbounds y[i,j] = x[i-1-w]
    end
    return y
end

function sliding_window_gpu!(x, w::Int, y)
    @assert size(y,1) == size(x,1)
    @assert size(y,2) == 2*w+1
    
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    sx = gridDim().x * blockDim().x
    sy = gridDim().y * blockDim().y
        
    for i = ix:sx:size(x,1), j = iy:sy:2*w+1
        @inbounds y[i,j] = x[i-1-w]
    end
    return
end

function bench_sliding_window_gpu!(x,w,y;blockdims=(32,8))
    n = size(x,1)
    m = 2*w+1
    griddims = cuda_grid((n,m), blockdims)
    CUDA.@sync begin
        @cuda threads=blockdims blocks=griddims  sliding_window_gpu!(x, w, y)
    end
end;


n = 360
m = 360
x = collect(1:n)
y = zeros(n,2*m+1)

x_ = CuArray(x)
y_ = CuArray(y)

blockdims = (16,16)
griddims = cuda_grid((n,m), blockdims)
@cuda threads=blockdims blocks=griddims sliding_window!(x_, m, y_);
@btime sliding_window_cpu!(x, m, y) samples=3 evals=3;
@btime bench_sliding_window_gpu!($x_, $m, $y_; blockdims=$blockdims) samples=3 evals=3;

  648.991 μs (0 allocations: 0 bytes)
  21.635 μs (21 allocations: 1.14 KiB)


In [ ]:
sliding_window_cpu!(x, m, y)